In [ ]:
import pandas as pd 
import re
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
stop_words.add('none')
stemmer = SnowballStemmer("english")
data=pd.read_csv(r'C:\Users\x2125\Desktop\full.csv')
data=data.drop(columns=['rewards'])
data=data.dropna(subset=['reward'])
data[['reward']].describe()
data['reward'].hist()

In [ ]:
data.isna().sum()
data.info()
data=data[data['status'].isin(['DONE','TODO','IN_REVIEW','IN_PROGESS'])]
data['due_date']=data['due_date'].transform(lambda x:'2024-01-01' if x=='None' else x)
#remove punctuations
data['name']=data['name'].str.lower().str.replace('[^\w\s]','')
data['description']=data['description'].apply(lambda x:re.sub('[^A-Za-z0-9]+',' ', str(x))).str.lower().str.split()
data['stemmed_description'] =data['description'].apply(lambda x: [i  for i in x if i.lower() not in stop_words])
data['stemmed_description'] =data['stemmed_description'].apply(lambda x: [stemmer.stem(y) for y in x])


In [306]:
data['stemmed_description']=data['stemmed_description'].apply(lambda x:' '.join(x))
data=data.drop(['description'],axis=1)


In [ ]:
grouped_by_status =data.groupby(['status'])
grouped_by_status.boxplot()
group_by_priority=data.groupby(['priority'])
group_by_priority.boxplot()
data.plot.scatter(x='creat',y='reward')

In [307]:
data['creat']=data['creat'].transform(lambda x:x[:10])
data['due_date']=data['due_date'].transform(lambda x:x[:10])
data['creat']=data['creat'].transform(lambda x:pd.to_datetime(x) if x !='None' else x)
data['due_date']=data['due_date'].transform(lambda x:pd.to_datetime(x))
data['due_date']=data['due_date'].dt.normalize()
data['time_to_due_date']=data['due_date']-data['creat']
data['time_to_due_date']=pd.to_numeric(data['time_to_due_date'])
data=data[data['time_to_due_date']>0]


In [308]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
target=data[['reward']]
variables=data.drop(columns=['reward'])
X_train,X_test,y_train,y_test=train_test_split(variables,target)

In [312]:
preprocessor=ColumnTransformer([('drop_columns','drop',['id','audit_log','subtasks','stemmed_description','tags','review','comments','name']),('OneHot',OneHotEncoder(handle_unknown='ignore'),['creator','dao','priority','status','assignees']),('StandardScalar',StandardScaler(),['time_to_due_date'])])

In [316]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
clf = Pipeline(
    steps=[("preprocessor", preprocessor),('classifier',RandomForestRegressor(max_depth=10,verbose=1))])

In [317]:
clf.fit(X_train, y_train)

c:\Users\x2125\anaconda3\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.4s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('drop_columns', 'drop',
                                                  ['id', 'audit_log',
                                                   'subtasks',
                                                   'stemmed_description',
                                                   'tags', 'review', 'comments',
                                                   'name']),
                                                 ('OneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['creator', 'dao', 'priority',
                                                   'status', 'assignees']),
                                                 ('StandardScalar',
                                                  StandardScaler(),
                                                  ['time_to_due_date'])])),
                ('classifier', RandomForestRegressor(max_depth=10, verbose=1))])

In [319]:
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.852


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [321]:
target.describe()

,reward
count,6893.000000
mean,193.956297
std,560.656375
min,0.000000
25%,0.295200
50%,0.999800
75%,1.000000
max,1850.530000
